# 1章 イントロダクション

## 1.1 バージョンについて

Ansibleのバージョン

> 原書は1.8.4で書かれていますが、翻訳時に2.0.0.4で動作検証しています。

## 1.2 Ansibleが役立つこと

Ansibleは、デプロイメントにも、設定管理にも使える。

デプロイメントのオーケストレーションとして、複数サーバーに対して処理を行えるようにできている。

サーバーのプロビジョニング（作成?）もサポートしている。

## 1.3 Ansibleの動作

AnsibleのスクリプトはYAML形式でplaybookと呼ぶ。順序づけされたタスクのリストを記述する。

playbookの実行はansible-playbookコマンド

```
$ ansible-playbook webservers.yml
```

Ansibleは各サーバーにSSH接続を行って処理を実行する。

## 1.4 Ansibleがすばらしい理由

## 1.4.1 読みやすい構文

## 1.4.2 リモートホストへのインストールが不要

## 1.4.3 プッシュベース

* Ansibleはリモートホストにエージェント不要なプッシュベース。
* Chef, Puppetはリモートホストにエージェントをインストールして使うプルベース。


* プッシュベース
  * メリット
    * エージェントレス
    * 実行タイミングを自分でコントロールしやすい
  * デメリット
    * キックする仕組みが必要


* プルベース
  * メリット
    * 自律的なシステム（新しいサーバーが追加されたらすぐに対応できる）
  * デメリット
    * エージェントが必要


Ansibleにもプルベースで実行できるansible-pullというツールがある。

## 1.4.4 スケールダウン

Ansibleは1台のノードを設定するためでも使える。playbookを1つ書くだけで実行できる。

## 1.4.5 組み込みモジュール

モジュールを使えば、パッケージのインストール、サービスの再起動、設定ファイルのコピーといったタスクを実行できます。

Ansibleのモジュールは宣言的で冪等。

収斂（しゅうれん）: 収束? 何度も実行する度に希望する状態へ近づけていくこと。  
Ansibleは1度の実行で希望する状態にできるようになっているため、収斂という考え方には実際には当てはまらない。

Ansibleの作者(Michael DeHaan)が収斂についてどのように考えているのか、以下を参照。

OT: Idempotence, convergence, and other silly fancy words we use too often  
https://groups.google.com/forum/#!msg/ansible-project/WpRblldA2PQ/lYDpFjBXDlsJ

## 1.4.6 抽象化層の薄さ

Ansibleはパッケージマネージャを抽象化していない。yumはyumモジュール、aptはaptモジュールを使う。これは新しい抽象概念を学ばなくてよくするため。

playbookは異なるコンテキストで再利用されることを意図したものではない。汎用的なplaybookを再利用するよりも、その組織のためのplaybookを書くほうがうまくいく。

## 1.5 Ansibleはシンプルすぎる?

> Ansibleはシェルスクリプトよりもはるかに多くの機能を提供しています。

> パフォーマンスの最適化のためにSSHマルチプレキシングを利用しており、世の中には数千ノードをAnsibleで管理している人々がいます。

## 1.6 事前に必要な知識

Linuxシステム管理の基本

AnsibleはPythonでできているが、Pythonを知らなくてもAnsibleを使うことはできる。

YAMLファイルフォーマットとJinja2テンプレート言語は知っているひつようがあるが、どちらも簡単。

## 1.7 本書で取り上げないこと

公式のAnsibleモジュール群の詳細。公式のドキュメントは充実している。

## 1.8 Ansibleのインストール

```
$ brew install ansible
```

In [1]:
ansible --version

ansible 2.1.0.0
  config file = 
  configured module search path = Default w/o overrides


## 1.9 テスト用サーバーのセットアップ

## 1.9.1 Vagrantを使ったテストサーバーのセットアップ

In [2]:
mkdir playbooks
cd playbooks
vagrant init ubuntu/trusty64

A `Vagrantfile` has been placed in this directory. You are now
ready to `vagrant up` your first virtual environment! Please read
the comments in the Vagrantfile as well as documentation on
`vagrantup.com` for more information on using Vagrant.


```
$ vagrant up
$ vagrant ssh
```

In [5]:
vagrant ssh-config

Host default
  HostName 127.0.0.1
  User vagrant
  Port 2222
  UserKnownHostsFile /dev/null
  StrictHostKeyChecking no
  PasswordAuthentication no
  IdentityFile /Users/speg03/.ghq/github.com/speg03/notebooks/ansible_up_and_running/playbooks/.vagrant/machines/default/virtualbox/private_key
  IdentitiesOnly yes
  LogLevel FATAL



## 1.9.2 テストサーバーのことをAnsibleに知らせる

In [8]:
cat hosts

testserver ansible_ssh_host=127.0.0.1 ansible_ssh_port=2222 ansible_ssh_user=vagrant ansible_ssh_private_key_file=.vagrant/machines/default/virtualbox/private_key

サーバーへの接続確認は`ansible`コマンドを使う。アドホックな一度限りの処理に使われる。pingモジュールはSSHで接続できるかどうかを確認するだけ。

In [14]:
ansible testserver -i hosts -m ping

testserver | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


## 1.9.3 ansible.cfgによる簡略化

Ansibleの設定ファイルは以下の順序で探される。

1. 環境変数のANSIBLE_CONFIG
1. カレントディレクトリのansible.cfg
1. HOMEのansible.cfg
1. /etc/ansible/ansible.cfg

In [11]:
cat ansible.cfg

[defaults]
hostfile = hosts
remote_user = vagrant
private_key_file = .vagrant/machines/default/virtualbox/private_key
host_keu_checking = False


ansible.cfgでデフォルト値を設定すると以下のようにhostsファイルはシンプルになる。

In [12]:
cat hosts

testserver ansible_ssh_host=127.0.0.1 ansible_ssh_port=2222


In [13]:
ansible testserver -m ping

testserver | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


In [15]:
ansible testserver -m ping -vvvv

Using /Users/speg03/.ghq/github.com/speg03/notebooks/ansible_up_and_running/playbooks/ansible.cfg as config file
Loaded callback minimal of type stdout, v2.0
<127.0.0.1> ESTABLISH SSH CONNECTION FOR USER: vagrant
<127.0.0.1> SSH: EXEC ssh -C -vvv -o ControlMaster=auto -o ControlPersist=60s -o Port=2222 -o 'IdentityFile=".vagrant/machines/default/virtualbox/private_key"' -o KbdInteractiveAuthentication=no -o PreferredAuthentications=gssapi-with-mic,gssapi-keyex,hostbased,publickey -o PasswordAuthentication=no -o User=vagrant -o ConnectTimeout=10 -o ControlPath=/Users/speg03/.ansible/cp/ansible-ssh-%h-%p-%r 127.0.0.1 '/bin/sh -c '"'"'( umask 77 && mkdir -p "` echo $HOME/.ansible/tmp/ansible-tmp-1469456479.25-64495621789304 `" && echo ansible-tmp-1469456479.25-64495621789304="` echo $HOME/.ansible/tmp/ansible-tmp-1469456479.25-64495621789304 `" ) && sleep 0'"'"''
<127.0.0.1> PUT /var/folders/pb/60xc31qn0ql95fv8cfcmqsjc0000gn/T/tmpoLo9nC TO /home/vagrant/.ansible/tmp/ansible-tmp-1469456479

commandモジュールを使って任意のコマンドを実行することができる。`-a`で実行するコマンドを渡してやる。

In [16]:
ansible testserver -m command -a uptime

testserver | SUCCESS | rc=0 >>
 14:23:48 up 46 min,  1 user,  load average: 0.00, 0.01, 0.05



commandモジュールは指定しなくてもよい?

In [17]:
ansible testserver -a uptime

testserver | SUCCESS | rc=0 >>
 14:23:58 up 47 min,  1 user,  load average: 0.00, 0.01, 0.05



In [18]:
ansible testserver -a "tail /var/log/dmesg"

testserver | SUCCESS | rc=0 >>
[    9.504582] type=1400 audit(1469453816.451:12): apparmor="STATUS" operation="profile_replace" profile="unconfined" name="/usr/lib/connman/scripts/dhclient-script" pid=1002 comm="apparmor_parser"
[    9.504737] type=1400 audit(1469453816.451:13): apparmor="STATUS" operation="profile_replace" profile="unconfined" name="/usr/lib/connman/scripts/dhclient-script" pid=1002 comm="apparmor_parser"
[    9.782122] type=1400 audit(1469453816.723:14): apparmor="STATUS" operation="profile_load" profile="unconfined" name="/usr/sbin/tcpdump" pid=1004 comm="apparmor_parser"
[    9.832269] vboxvideo: Unknown symbol drm_open (err 0)
[    9.832275] vboxvideo: Unknown symbol drm_poll (err 0)
[    9.832278] vboxvideo: Unknown symbol drm_pci_init (err 0)
[    9.832332] vboxvideo: Unknown symbol drm_ioctl (err 0)
[    9.832335] vboxvideo: Unknown symbol drm_mmap (err 0)
[    9.832338] vboxvideo: Unknown symbol drm_pci_exit (err 0)
[    9.832341] vboxvideo: Unknown symbol drm

In [19]:
ansible testserver -a "tail /var/log/syslog"

testserver | FAILED | rc=1 >>
tail: cannot open ‘/var/log/syslog’ for reading: Permission denied



`-s`フラグを渡せば`sudo`で`root`権限になる。

In [20]:
ansible testserver -s -a "tail /var/log/syslog"

testserver | SUCCESS | rc=0 >>
Jul 25 13:37:05 vagrant-ubuntu-trusty-64 ec2: 2048 ab:1e:93:bc:f4:6b:3b:42:58:26:9b:9f:bf:4f:18:9c  root@vagrant-ubuntu-trusty-64 (RSA)
Jul 25 13:37:05 vagrant-ubuntu-trusty-64 ec2: -----END SSH HOST KEY FINGERPRINTS-----
Jul 25 13:37:05 vagrant-ubuntu-trusty-64 ec2: #############################################################
Jul 25 13:37:05 vagrant-ubuntu-trusty-64 kernel: [   16.348840] init: cloud-final main process (1364) terminated with status 2
Jul 25 13:37:05 vagrant-ubuntu-trusty-64 puppet-agent[1266]: Could not request certificate: getaddrinfo: Name or service not known
Jul 25 13:37:10 vagrant-ubuntu-trusty-64 ntpdate[1334]: adjust time server 91.189.91.157 offset -0.001083 sec
Jul 25 14:00:40 vagrant-ubuntu-trusty-64 ansible-basic.py: Invoked with data=None
Jul 25 14:01:31 vagrant-ubuntu-trusty-64 ansible-basic.py: Invoked with data=None
Jul 25 14:17:01 vagrant-ubuntu-trusty-64 CRON[2406]: (root) CMD (   cd / && run-parts --report /etc/cron.ho

aptモジュールを使ってnginxをインストールする。

In [21]:
ansible testserver -s -m apt -a name=nginx

testserver | SUCCESS => {
    "cache_update_time": 0, 
    "cache_updated": false, 
    "changed": true, 
    "stderr": "", 
    "stdout": "Reading package lists...\nBuilding dependency tree...\nReading state information...\nThe following extra packages will be installed:\n  libxslt1.1 nginx-common nginx-core\nSuggested packages:\n  fcgiwrap nginx-doc\nThe following NEW packages will be installed:\n  libxslt1.1 nginx nginx-common nginx-core\n0 upgraded, 4 newly installed, 0 to remove and 0 not upgraded.\nNeed to get 493 kB of archives.\nAfter this operation, 1795 kB of additional disk space will be used.\nGet:1 http://archive.ubuntu.com/ubuntu/ trusty/main libxslt1.1 amd64 1.1.28-2build1 [145 kB]\nGet:2 http://archive.ubuntu.com/ubuntu/ trusty-updates/main nginx-common all 1.4.6-1ubuntu3.5 [18.1 kB]\nGet:3 http://archive.ubuntu.com/ubuntu/ trusty-updates/main nginx-core amd64 1.4.6-1ubuntu3.5 [325 kB]\nGet:4 http://archive.ubuntu.com/ubuntu/ trusty-updates/main nginx all 1.4.6-1ubuntu3

## 1.10 今後に向けて